In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import math

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split


2023-11-16 23:43:13.340444: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-16 23:43:13.360332: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 23:43:13.360356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 23:43:13.361019: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 23:43:13.364588: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-16 23:43:13.365032: I tensorflow/core/platform/cpu_feature_guard.cc:1

# Load Data

First step is to ingest all the data we have available and merge them into a flattened datastructure containing all measurements. Indexes are ignored and rewritten to allow all readings to be added to the DF:

In [2]:
# parse the labels.csv
labels = pd.read_csv('labels.csv', index_col=0)
labels = labels.sort_values('id')

# grab filenames from the data directory
filenames = os.listdir('data')
filenames.sort()

dataframes = []

# parse and concatenate all csv files into df
for filename in filenames:
  if filename.endswith('.csv'):
    batch = pd.read_csv(os.path.join('data',filename), index_col=0)
    batch['batch'] = int(filename.replace('.csv', ''))
    dataframes.append(batch)

df = pd.concat(dataframes, ignore_index=True)

# clean up original dataframes
del dataframes

# add label column (if it is not already available)
if (not 'label' in df.columns):
  df = df.merge(labels, left_on=["batch"], right_on=["id"])


In [3]:
def time_to_float(inputstr):
  hours, minutes, seconds = map(float, inputstr.split(':'))

  # return hours * 3600 + minutes * 60 + seconds
  # this is sufficient because hours should always be 0
  return minutes * 60 + seconds

if (not df['sensorid'].dtype == 'int'):
  df['sensorid'] = df['sensorid'].astype('int')
if (not df['label'].dtype == 'category'):
  df['label'] = df['label'].astype('category')
if (not df['zeit'].dtype == 'float64'):
  df['zeit'] = df['zeit'].apply(time_to_float)

# print(df[:10])
# print(labels[:10])


# Test Data Preprocessing

In [4]:
SEQUENCE_LENGTH = 64

sequences = []
sequence_labels = []

for batch, readings in df.groupby('batch'):
  readings = readings.sort_values('zeit')
  for i in range(0, len(readings) - SEQUENCE_LENGTH, SEQUENCE_LENGTH):
    sequence = readings.iloc[i:i + SEQUENCE_LENGTH]
    sequences.append(sequence[['zeit', 'sensorid', 'messwert']].values)
    sequence_labels.append(sequence['label'].values[0])

sequences = np.array(sequences)
sequence_labels = np.array(sequence_labels)

X_train, X_test, y_train, y_test = train_test_split(sequences, sequence_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


# Modelling

In [9]:
# set a bunch of constants that help configure the model and training
BATCH_SIZE = 64
LSTM_UNITS = 256

CHECKPOINT_PATH = '.checkpoints/cp-{epoch:03d}.ckpt'
# CHECKPOINT_DIR = os.path.dirname(CHECKPOINT_PATH)

N_BATCHES = math.ceil(len(X_train) / BATCH_SIZE)
num_features = X_train.shape[2]
num_classes = len(labels['label'].unique())

model = Sequential()
model.add(LSTM(LSTM_UNITS, input_shape=(SEQUENCE_LENGTH, num_features)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training
cp_cb = ModelCheckpoint(filepath=CHECKPOINT_PATH, save_weights_only=True, save_freq=8*N_BATCHES)
stp_cb = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, min_delta=1e-4, start_from_epoch=32, verbose=1)

model.fit(X_train, y_train, epochs=256, batch_size=BATCH_SIZE, callbacks=[cp_cb, stp_cb], validation_data=(X_val, y_val))

# Save Model
model.save(f'models/lstm_{LSTM_UNITS}-seq_{SEQUENCE_LENGTH}-batch_{BATCH_SIZE}.keras')



Epoch 1/256
618/618 [==============================] - 43s 68ms/step - loss: 1.1046 - accuracy: 0.3452 - val_loss: 1.1011 - val_accuracy: 0.3351
Epoch 2/256
618/618 [==============================] - 41s 66ms/step - loss: 1.1013 - accuracy: 0.3447 - val_loss: 1.0976 - val_accuracy: 0.3373
Epoch 3/256
618/618 [==============================] - 41s 67ms/step - loss: 1.0993 - accuracy: 0.3497 - val_loss: 1.0997 - val_accuracy: 0.3671
Epoch 4/256
618/618 [==============================] - 41s 66ms/step - loss: 1.0969 - accuracy: 0.3614 - val_loss: 1.0941 - val_accuracy: 0.3583
Epoch 5/256
618/618 [==============================] - 41s 66ms/step - loss: 1.0958 - accuracy: 0.3642 - val_loss: 1.0927 - val_accuracy: 0.3645
Epoch 6/256
618/618 [==============================] - 41s 67ms/step - loss: 1.0946 - accuracy: 0.3677 - val_loss: 1.0909 - val_accuracy: 0.3783
Epoch 7/256
618/618 [==============================] - 40s 66ms/step - loss: 1.0931 - accuracy: 0.3712 - val_loss: 1.0916 - val_ac

# Evaluation

Best current model `lstm_100-seq_128-batch_64` (74.74%)

In [10]:
print(model.predict(X_test[:5]))
loss, acc = model.evaluate(X_test, y_test, verbose=2)

print("Model accuracy: {:5.2f}%".format(100 * acc))


1/1 [==============================] - 0s 214ms/step
[[4.8466197e-01 3.7249166e-01 1.4284633e-01]
 [1.8934818e-01 8.0996621e-01 6.8551704e-04]
 [3.5952237e-01 6.3849515e-01 1.9824610e-03]
 [3.5528547e-01 5.3132200e-01 1.1339250e-01]
 [7.2025174e-01 2.3314826e-01 4.6599906e-02]]
386/386 - 6s - loss: 0.6048 - accuracy: 0.7063 - 6s/epoch - 17ms/step
Model accuracy: 70.63%
